In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from datetime import datetime

In [3]:
sc = SparkSession.builder.appName("abcd").config("master", "local").getOrCreate()

25/11/10 18:49:32 WARN Utils: Your hostname, shasankperiwal resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlp0s20f3)
25/11/10 18:49:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 18:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
org_df = sc.read.csv("../assets/organizations.csv", header=True, inferSchema=True)
org_df.show()
org_df.printSchema()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+
|    1|E84A904909dF528|          Liu-Hoover|http://www.day-ha...|      Western Sahara|Ergonomic zero ad...|   1980|   Online Publishing|               6852|
|    2|AAC4f9aBF86EAeF|       Orr-Armstrong|https://www.chapm...|             Algeria|Ergonomic radical...|   1970|     Import / Export|               7994|
|    3|ad2eb3C8C24DB87|           Gill-Lamb|     http://lin.com/|       Cote d'Ivoire|Programmable inte...|   2005|   Apparel / Fashion|               5105|
|    4|D76BB12E5eE165B|         Bauer-Weiss|https://gilles

In [ ]:
# org_df.printSchema()
window = Window.partitionBy('Country').orderBy(col('founded'))
rows_df = org_df.withColumn('rowsBetween', row_number().over(window.rowsBetween(Window.unboundedPreceding, Window.currentRow)))
                    # .withColumn('rangeBetween', sum().over(window.rangeBetween(Window.unboundedPreceding, Window.currentRow)))

rows_df.select(col('country'), col('founded'), col('rowsBetween')).show()
# rows_df.select(col('country'), col('Number of employees'), col('rangeBetween')).show()

+--------------+-------+-----------+
|       country|founded|rowsBetween|
+--------------+-------+-----------+
|   Afghanistan|   1985|          1|
|   Afghanistan|   1990|          2|
|   Afghanistan|   1991|          3|
|       Albania|   1972|          1|
|       Albania|   1972|          2|
|       Algeria|   1970|          1|
|       Algeria|   1977|          2|
|       Algeria|   1981|          3|
|       Algeria|   1998|          4|
|       Algeria|   2021|          5|
|       Algeria|   2021|          6|
|American Samoa|   1971|          1|
|American Samoa|   1972|          2|
|American Samoa|   1972|          3|
|American Samoa|   1975|          4|
|American Samoa|   1980|          5|
|American Samoa|   1983|          6|
|American Samoa|   1989|          7|
|American Samoa|   1996|          8|
|       Andorra|   1980|          1|
+--------------+-------+-----------+
only showing top 20 rows



In [ ]:
data = [
    (1, datetime.strptime('2022-01-01', '%Y-%m-%d').date(), 'entire home,private room'),
    (2, datetime.strptime('2022-01-02', '%Y-%m-%d').date(), 'entire home,shared room'),
    (3, datetime.strptime('2022-01-02', '%Y-%m-%d').date(), 'private room,shared room'),
    (4, datetime.strptime('2022-01-03', '%Y-%m-%d').date(), 'private room')
]

struct = StructType([
    StructField("id", IntegerType()),
    StructField("date", DateType()),
    StructField("rules", StringType())
])

df = sc.createDataFrame(data=data, schema=struct)

df.show(truncate=False)

In [ ]:
odf = df.select("id", "date", explode(split("rules",",")))

In [ ]:
odf = odf.groupBy(col("id")).agg(count("*").alias("cnt"))
odf.show()

In [ ]:
odf = odf.filter(odf.cnt > 1)
odf.show()

In [ ]:
columns = ['emp_id', 'designation']
# Create DataFrame for 2020 and
emp_2020_data = [(1, 'Trainee'), (2, 'Developer'), (3, 'Senior Developer'), (4, 'Manager')]
emp_2021_data = [(1, 'Developer'), (2, 'Developer'), (3, 'Manager'), (5, 'Trainee')]

emp_2020_df = sc.createDataFrame(emp_2020_data, columns)
emp_2021_df = sc.createDataFrame(emp_2021_data, columns)

emp_2020_df.show()
emp_2021_df.show()

In [ ]:
# Show status on each employee from year 2020 to 2021 if changed
# If new - NEW; If re signed - RESIGNED; If designation changed - PROMOTED
emp = emp_2020_df.join(emp_2021_df, on="emp_id", how="fullouter").withColumn("Status", when(emp_2020_df.designation != emp_2021_df.designation, "PROMOTED").when(emp_2020_df.designation == emp_2021_df.designation, "SAME").when(emp_2020_df.designation.isNull(), "NEW").when(emp_2021_df.designation.isNull(), "RESIGNED")).select("emp_id", "status")
emp.show()

In [ ]:
def read_csv(path):
    return sc.read.csv(path=path, inferSchema=True, header=True)

In [ ]:
orgsdf = read_csv("assets/organizations.csv")
orgsdf.show(5)

In [ ]:
orgsdf.select("Name").distinct().count()
# orgsdf.select("Name").count()

In [ ]:
# Query to print the first 3 characters of FIRST_NAME from Student table.
orgsdf.select(substring("Name", 0, 3)).distinct().count()

In [ ]:
# Query to find the position of alphabet (‘a’) int the first name column ‘Shivansh’ from Student table.
orgsdf.select("Name", instr(col("Name"), 'a').alias('index')).show()

In [ ]:
# Query that fetches the unique values of MAJOR Subjects from Student table and print its length.
orgsdf.select("Name").distinct().withColumn("length", length(col("Name"))).show(5)

In [ ]:
# Query to print FIRST_NAME from the Student table after replacing ‘a’ with ‘A’.
orgsdf.select("Name").withColumn("NAme", regexp_replace(col("Name"), 'a', 'A')).show(5)

In [ ]:
# Query to print the FIRST_NAME and LAST_NAME from Student table into single column COMPLETE_NAME.
orgsdf.select(concat(orgsdf.Name,lit(" | "),orgsdf.Country).alias("Name | Country")).show(5, truncate=False)

In [ ]:
# Query to print all Student details order by FIRST_NAME Ascending and MAJOR Subject descending .
orgsdf.select("Name", "Country").orderBy(desc("Name"), "Country").show(5, truncate=False)

In [ ]:
# Query to print details of the Students excluding FIRST_NAME as ‘Prem’ and ‘Shivansh’.
orgsdf.select("Name", "Country").orderBy(desc("Name"), "Country").filter(~col("Country").isin(["Iran", "Costa Rica"])).show(5, truncate=False)

In [ ]:
# Query to print details of the Students whose FIRST_NAME ends with ‘a’ and contains five alphabets.
orgsdf.select("Name").filter(col("Name").like('____g%')).show(5, truncate=False)

In [ ]:
# Query to fetch the count of Students having Major Subject ‘Computer Science’.
orgsdf.select("Country").filter(col("Country") == "India").count()

In [ ]:
# Query to fetch the no. of Students for each MAJOR subject in the descending order.
orgsdf.groupBy("Country").agg(count("*").alias("cnt")).orderBy(desc("cnt")).show(10, truncate=False)

In [ ]:
# Query to show only odd rows from Student table.
orgsdf.select("Name","Website","Country","Description","Founded", row_number().over(Window.orderBy("Founded")).alias("rn")).filter("rn%2 != 0").show(10, truncate=False)

In [ ]:
# Query to fetch the first 50% records from a table.
orgsdf.select("Name","Website","Country","Description","Founded", percent_rank().over(Window.orderBy("Founded")).alias("rn")).show()

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [ ]:
sc = SparkSession.builder.appName('master').config('master', 'local').getOrCreate()

In [ ]:
df = sc.read.csv("../assets/currency.csv")

In [ ]:
df.explain("formatted")